# Neural Creative Transfer

We implement an modification of Gatys, Ecker and Bethge's [neural style transfer algorithm](http://arxiv.org/abs/1508.06576) for our creative assets.

The algorith uses a VGG-19 network to separate and recombine content of an creative and style of a website, and therefore provides a system for creating native creative for advertising. 

In [2]:
import os
import sys

import numpy as np
import scipy.io
import scipy.misc
import tensorflow as tf 

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image

%matplotlib inline

In [4]:
def setup_model_from_file(path, image_height, image_width, color_channels):
    """
    We start with the pre-trained VGG model from the paper "Very Deep Convolutional Networks for Large-Scale 
    Visual Recognition". We  strip out all the conv, relu and pool layers from the VGG-19 network and replace
    the maxpool layers with average pool layers as suggested by the authors in the "A Neural Algorithm of 
    Artistic Style" paper. Here are the configuration for our modified VGG network (layer, filter height,
    filter width, input channels, output channels):
        conv1_1  :  3  3  3    64
        conv1_2  :  3  3  64   64 
        avgpool  :
        conv2_1  :  3  3  64   128  
        conv2_2  :  3  3  128  128 
        avgpool  :
        conv3_1  :  3  3  128  256 
        conv3_2  :  3  3  256  256
        conv3_3  :  3  3  256  256
        conv3_4  :  3  3  256  256
        avgpool  :
        conv4_1  :  3  3  256  512  
        conv4_2  :  3  3  512  512
        conv4_3  :  3  3  512  512
        conv4_4  :  3  3  512  512
        avgpool  :
        conv5_1  :  3  3  512  512 
        conv5_2  :  3  3  512  512 
        conv5_3  :  3  3  512  512 
        conv5_4  :  3  3  512  512 
        avgpool  :    
    """
    
    # Strip out the layers from the VGG-19 model 
    vgg = scipy.io.loadmat(path)['layers'] 
    
    def _weights(layer):
        """
        Extract the weights and bias at a given layer.
        """
        W = vgg[0][layer][0][0][2][0][0]
        b = vgg[0][layer][0][0][2][0][1]
        return W, b

    def _conv2d(prev_layer, layer):
        """
        Construct the conv2d layer from the VGG model.
        """
        W, b = _weights(layer)
        W = tf.constant(W)
        b = tf.constant(np.reshape(b, (b.size)))
        return tf.nn.conv2d(prev_layer, filter=W, strides=[1, 1, 1, 1], padding='SAME') + b

    def _avg_pool(prev_layer):
        """
        Return the average pooling layer.
        """
        return tf.nn.avg_pool(prev_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Set up the modifiled VGG network.  
    network = {}
    network['input']    = tf.Variable(np.zeros((1, image_height, image_width, color_channels)), dtype = 'float32')
    network['conv1_1']  = tf.nn.relu(_conv2d(network['input'], 0))
    network['conv1_2']  = tf.nn.relu(_conv2d(network['conv1_1'], 2))    
    network['avgpool1'] = _avgpool(network['conv1_2'])
    network['conv2_1']  = tf.nn.relu(_conv2d(network['avgpool1'], 5))
    network['conv2_2']  = tf.nn.relu(_conv2d(network['conv2_1'], 7))
    network['avgpool2'] = _avgpool(network['conv2_2'])
    network['conv3_1']  = tf.nn.relu(_conv2d(network['avgpool2'], 10))
    network['conv3_2']  = tf.nn.relu(_conv2d(network['conv3_1'], 12))
    network['conv3_3']  = tf.nn.relu(_conv2d(network['conv3_2'], 14))
    network['conv3_4']  = tf.nn.relu(_conv2d(network['conv3_3'], 16))
    network['avgpool3'] = _avgpool(network['conv3_4'])
    network['conv4_1']  = tf.nn.relu(_conv2d(network['avgpool3'], 19))
    network['conv4_2']  = tf.nn.relu(_conv2d(network['conv4_1'], 21))
    network['conv4_3']  = tf.nn.relu(_conv2d(network['conv4_2'], 23))
    network['conv4_4']  = tf.nn.relu(_conv2d(network['conv4_3'], 25))
    network['avgpool4'] = _avgpool(network['conv4_4'])
    network['conv5_1']  = tf.nn.relu(_conv2d(network['avgpool4'], 28))
    network['conv5_2']  = tf.nn.relu(_conv2d(network['conv5_1'], 30))
    network['conv5_3']  = tf.nn.relu(_conv2d(network['conv5_2'], 32))
    network['conv5_4']  = tf.nn.relu(_conv2d(network['conv5_3'], 34))
    network['avgpool5'] = _avgpool(network['conv5_4'])
    return network